In [1]:
from src.spark import Spark
import src.tweet_volume as funcs
import matplotlib.pyplot as plt
from pyspark.sql import functions as fs
import os 
import pandas as pd
from src.plotting import double_plot

In [2]:
spark = Spark('load', 'spark://pc2-060-l.cs.st-andrews.ac.uk:7077')
sess = spark.session()

In [3]:
df = funcs.load_dataframe(sess, funcs.default_loc, funcs.schema)
df.first()

Row(text=u'RT @VinnyLingham2x: #Bitcoin needs less developers, and more incumbents and intermediaries! https://t.co/iia0k7i09B', entities=Row(hashtags=[Row(indices=[20, 28], text=u'Bitcoin')]), created_at=u'Thu Oct 26 06:26:17 +0000 2017', favourite_count=None, retweet_count=0, id_str=u'923435597242478592', user=Row(followers_count=59, utc_offset=None))

In [4]:
df2 = funcs.parse_timestmap(df)




In [ ]:
volume_df = df2.withColumn("date", fs.date_format("timestamp", "yyyy-MM-dd")).groupBy("date").count() .toPandas()

volume_df

In [8]:

crix = pd.read_csv(os.path.join("/cs/home/ls99/PycharmProjects/Spark/data/crypto", "crix.csv"), index_col=0)
crix.index = pd.to_datetime(crix.index)


In [ ]:
volume_df.dropna()
volume_df.set_index('date', inplace=True)
volume_df.index = pd.to_datetime(volume_df.index)

crix_and_vol = crix
crix_and_vol['volume'] = volume_df['count']
crix_and_vol_nona = crix_and_vol.dropna()

In [15]:
crix_and_vol_nona.corr()

,price,volume
price,1.000000,0.585943
volume,0.585943,1.000000


In [39]:
scaled_price = crix_and_vol_nona.price * (crix_and_vol_nona.volume.iloc[0] / crix_and_vol_nona.price.iloc[0])
double_plot([crix_and_vol_nona.price, crix_and_vol_nona.volume], ['Index Price', 'Tweets'], ['Date', 'Price', 'Volume'], "Price vs Tweet Volume", crix_and_vol_nona.index.tolist())

In [42]:
crix_and_vol_nona.corr().price.volume


0.58594334311745255

In [103]:
btc_df = df2.filter(fs.lower(df['text']).like("%bitcoin%") | fs.lower(df['text']).like("%btc%"))

In [ ]:
btc_df_vol = btc_df.withColumn("date", fs.date_format("timestamp", "yyyy-MM-dd")).groupBy("date").count().toPandas()

btc = pd.read_csv(os.path.join("/cs/home/ls99/PycharmProjects/Spark/data/crypto/KRAKEN_BTC_EUR.csv"), index_col=0)
btc.index = pd.to_datetime(btc.index)

btc_df_vol.dropna()
btc_df_vol.index = pd.to_datetime(btc_df_vol.index)

btc_and_vol = btc
btc_and_vol['price'] = btc_and_vol.weightedAverage
btc_and_vol['volume'] = btc_df_vol['count']
btc_and_vol_nona = btc_and_vol.dropna()

double_plot([btc_and_vol_nona.price, btc_and_vol_nona.volume], ['BTC Price', 'Tweets'], ['Date', 'Price', 'Volume'], "BTC Price vs Tweet Volume", btc_and_vol_nona.index.tolist())
btc_and_vol_nona.corr().price.volume

In [5]:
eth_df = df2.filter(fs.lower(df['text']).like("%ether%") | fs.lower(df['text']).like("%eth%") | fs.lower(df['text']).like("%ethereum%"))


In [ ]:
eth_df_vol = eth_df.withColumn("date", fs.date_format("timestamp", "yyyy-MM-dd")).groupBy("date").count().toPandas()

eth = pd.read_csv(os.path.abspath(os.path.join("/cs/home/ls99/PycharmProjects/Spark/data/crypto","KRAKEN_ETH_EUR.csv")), index_col=0)
eth.index = pd.to_datetime(eth.index)

eth_df_vol.dropna()
eth_df_vol.set_index('date', inplace=True)
eth_df_vol.index = pd.to_datetime(eth_df_vol.index)

eth_and_vol = eth
eth_and_vol['price'] = eth_and_vol.weightedAverage
eth_and_vol['volume'] = eth_df_vol['count']
eth_and_vol_nona = eth_and_vol.dropna()

double_plot([eth_and_vol_nona.price, eth_and_vol_nona.volume], ['ETH Price', 'Tweets'], ['Date', 'Price', 'Volume'], "ETH Price vs Tweet Volume", eth_and_vol_nona.index.tolist())

In [123]:
eth_df_vol

,date,count
1970-01-01 00:00:00.000000000,2017-10-20,33089
1970-01-01 00:00:00.000000001,2017-10-31,17956
1970-01-01 00:00:00.000000002,2017-11-01,37185
1970-01-01 00:00:00.000000003,2017-10-24,17504
1970-01-01 00:00:00.000000004,2017-10-22,38421
1970-01-01 00:00:00.000000005,2017-11-10,24914
1970-01-01 00:00:00.000000006,2017-11-23,27057
1970-01-01 00:00:00.000000007,2017-11-16,49582
1970-01-01 00:00:00.000000008,2017-11-14,45862
1970-01-01 00:00:00.000000009,2017-11-15,48066


In [125]:
eth_and_vol_nona.corr().price.volume

0.39946022045860696

In [6]:
from src.nlp.sentiment import SentimentAnalyser

Py4JJavaError: An error occurred while calling o152.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 61 in stage 2.0 failed 4 times, most recent failure: Lost task 61.3 in stage 2.0 (TID 179, 138.251.29.69, executor 16): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/cs/home/ls99/Downloads/spark-master/python/lib/pyspark.zip/pyspark/worker.py", line 229, in main
    process()
  File "/cs/home/ls99/Downloads/spark-master/python/lib/pyspark.zip/pyspark/worker.py", line 224, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/cs/home/ls99/Downloads/spark-master/python/lib/pyspark.zip/pyspark/worker.py", line 149, in <lambda>
    func = lambda _, it: map(mapper, it)
  File "<string>", line 1, in <lambda>
  File "/cs/home/ls99/Downloads/spark-master/python/lib/pyspark.zip/pyspark/worker.py", line 74, in <lambda>
    return lambda *a: f(*a)
  File "/cs/home/ls99/Downloads/spark-master/pyt